<p style="color: #00099B; font-size: 30px; font-weight: bold;">
The Wallet or the Ledger? How Inflation and Student Debt Drive Independent Voter Preferences
</p>

## 1. Introduction

### In the polarized landscape of American politics, **Independent voters**—those who do not align with either the Democratic or Republican parties—often determine the outcome of national elections. **Scenario C** of our analysis focuses on this pivotal group.  
> **Scenario C:** This scenario isolates Independent voters to examine how their unique preferences shift in response to economic pressures, without the strong partisan biases typically seen in Democrat or Republican voters.

### As the 2024 election approaches, **economic concerns have become central** to voter behavior. Among these concerns, rising prices and the burden of student loan repayment stand out as the financial stressors most frequently cited by households.

### This project investigates a targeted research question:

<p style="color: #00099B; font-size: 24px; font-weight: bold;"> 
What factors shape Independent voters’ favorability toward President Biden across economic, security, social, cultural, and foreign policy concerns?</p>

### The goal is to understand whether these economic pressures meaningfully influence political support within a group whose preferences are both less predictable and more electorally decisive.

<table>
  <tr>
    <td style="width: 250px;">
    <p align="center">
      <img src="Official_portrait_of_Vice_President_Joe_Biden.jpg" width="230">
      </p>
    </td>
    <td>

### Key Biden Policy Positions

- **Student Loan Relief**
  - Proposed cancelling federal student loans for eligible low– and middle–income borrowers.
  - Framed student debt as a barrier to upward mobility.
  - Part of the broader “Bidenomics” focus on reducing debt burdens.

- **Economic Stimulus & Household Support**
  - Implemented the American Rescue Plan Act of 2021.
  - Invested in infrastructure, clean energy, and public services.
  - Aimed to reduce financial vulnerability for working families.

- **Focus on Inequality & Social Equity**
  - Positioned loan forgiveness as a tool to reduce inequality.
  - Emphasized benefits for younger voters and disadvantaged communities.

    </td>
  </tr>
</table>


---
---

## 2. Data

### 2.1 Datasource

#### We utilize the **2024 ANES Pilot Study**, a **cross-sectional survey** of **1,909 U.S. adults** conducted in **March 2024**. We use these data because it offers **nationally representative**, **election-year data** with detailed measures of **economic stress** and **political attitudes**, allowing us to accurately isolate and analyze **Independent voters’** responses to **inflation** and **student debt**. The data was filtered to isolate **'Independent'**, **'Other'**, and **'Not Sure'** voters ($N = 393$).

In [7]:
import warnings
# Suppress the specific FutureWarning about use_inf_as_na
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [3]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(r"C:\Users\34502\OneDrive - Emory\Desktop\ECON320_Final_Project\anes_pilot_2024_20240319.csv")

# Filter for Independent/Non-Aligned Voters (Scenario C)
target_groups = ['Independent', 'Other', 'Not sure']
indies = df[df['pid3'].isin(target_groups)].copy()

print(f"Total Sample Size: {len(df)}")
print(f"Independent Sub-sample Size: {len(indies)}")

Total Sample Size: 1909
Independent Sub-sample Size: 704


### 2.2 Variable Definitions

| **Variable** | **Definition** | **Representation** |
|--------------|----------------|--------------------|
| **cand_biden** | Respondent’s favorability toward President Biden | 0–100 feeling thermometer |
| **infl_behav_count** | Index of inflation-related coping behaviors | Additive index (0–4): <br> • Borrowed to pay bills <br> • Cut spending <br> • Postponed purchases <br> • Used/dipped into savings |
| **stloan_amtowe** | Amount of student loan debt owed | Continuous (USD) |
| **Gender** | Gender of respondent | Binary: 0 = Male, 1 = Female |
| **Age** | Age in years | Continuous |
| **Race** | Racial background | Dummy-coded (White as reference): <br> • Black <br> • Hispanic <br> • Asian <br> • White/Other |
| **Education** | Highest educational attainment | Binary: College Graduate vs. Non-Graduate |
| **Ideology** | Political ideology | Categorical: Liberal, Moderate, Conservative |


In [5]:
# 1. dependent Variable: Clean Biden Thermometer
indies['cand_biden'] = pd.to_numeric(indies['cand_biden'], errors='coerce')

# 2. Key Independent Variable: Student Loans
# Convert to numeric, filling NaNs with 0 (assuming no answer = no debt for this analysis)
indies['stloan_amtowe'] = pd.to_numeric(indies['stloan_amtowe'], errors='coerce').fillna(0)

# 3. Key Independent Variable: Inflation Stress Index
# Map 'selected' to 1, others to 0
infl_cols = ['infl_behav_1', 'infl_behav_2', 'infl_behav_3', 'infl_behav_4']
for col in infl_cols:
    indies[col + '_num'] = indies[col].apply(lambda x: 1 if x == 'selected' else 0)

# Sum the behaviors to create a 0-4 index
indies['infl_behav_count'] = indies[[c + '_num' for c in infl_cols]].sum(axis=1)

# 4. Control Variables
indies['age'] = pd.to_numeric(indies['age'], errors='coerce')
indies['gender_bin'] = (indies['gender'] == 'Male').astype(int) # Male = 1
indies['college_grad'] = indies['educ'].isin(['4-year degree', 'Post-grad']).astype(int)

# Race Dummies (Reference = White)
indies['race_black'] = (indies['race'] == 'Black').astype(int)
indies['race_hispanic'] = (indies['race'] == 'Hispanic').astype(int)

# Ideology Dummies (Reference = Conservative/Other)
# We define Liberal if 'Liberal' appears in the text response
indies['ideo_liberal'] = indies['ideo5'].apply(lambda x: 1 if 'Liberal' in str(x) else 0)
indies['ideo_moderate'] = (indies['ideo5'] == 'Moderate').astype(int)

# Final Clean: Drop rows where the Dependent Variable (Biden Rating) is missing
reg_df = indies.dropna(subset=['cand_biden', 'age'])

---
---

## 3. Empirical Model

### To evaluate how inflation-related financial strain and student loan burdens shape attitudes toward President Biden, we estimate a series of linear regression models of the following general form:

## $$BidenRating = \beta_0 + \beta_1(Inflation) + \beta_2(StudentLoans) + \beta_3(Controls) + \epsilon$$

### 3.1 OLS Regression
#### We employ Ordinary Least Squares (OLS) regression to estimate the effect of individual economic stressors on Biden favorability. Our specification uses **robust standard errors** to account for heteroskedasticity and includes a full set of **demographic and ideological controls**.

#### - **Inflationᵢ** is measured using *infl_behav_count*, an additive index (0–4) capturing whether respondents borrowed to pay bills, cut spending, postponed purchases, or dipped into savings.  
#### - **StudentLoansᵢ** is measured by *stloan_amtowe*, a continuous variable indicating total student loan debt owed.  
#### - **Controlsᵢ** include gender, age, race, education, and ideology, all coded as binary, categorical, or continuous variables as appropriate.

#### The key parameters of interest are **β₁** and **β₂**, which indicate how inflation-related behaviors and student loan burdens, respectively, influence respondents’ overall favorability toward President Biden.

In [9]:
import statsmodels.api as sm

# Define the predictors (X) and outcome (y)
X = reg_df[['infl_behav_count', 'stloan_amtowe', 'gender_bin', 'age',
            'race_black', 'race_hispanic', 'college_grad',
            'ideo_liberal', 'ideo_moderate']]
X = sm.add_constant(X) # Add the intercept
y = reg_df['cand_biden']

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             cand_biden   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     13.99
Date:                Sun, 30 Nov 2025   Prob (F-statistic):           1.20e-20
Time:                        19:48:18   Log-Likelihood:                -3056.3
No. Observations:                 637   AIC:                             6133.
Df Residuals:                     627   BIC:                             6177.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               23.8283      4.629  

### **3.2 Robustness Checks**

#### To ensure our results were not driven by age-specific cohorts, we split the sample into voters under 40 and voters over 40. This allows us to see if student loans matter more to younger voters.



### Multicollinearity: VIF Table

In [ ]:
def calculate_vif(X):
    vif_df = pd.DataFrame()
    vif_df["Feature"] = X.columns
    vif_df["VIF"] = [
        variance_inflation_factor(X.values, i)
        for i in range(X.shape[1])
    ]
    return vif_df

vif_table = calculate_vif(X_clean.drop(columns=["const"]))
print(vif_table)



Table 2: Robustness Checks (Age Split)

                 Full Model Under 40 Over 40 
---------------------------------------------
const            23.83***   39.50*** 18.90** 
                 (4.63)     (10.26)  (9.25)  
infl_behav_count -2.15**    -2.47    -2.11*  
                 (0.95)     (1.50)   (1.21)  
stloan_amtowe    -0.00      0.00     -0.00   
                 (0.00)     (0.00)   (0.00)  
gender_bin       1.04       1.09     1.09    
                 (2.37)     (3.89)   (2.98)  
age              0.01       -0.34    0.06    
                 (0.07)     (0.33)   (0.14)  
race_black       19.15***   13.06**  21.54***
                 (3.81)     (5.30)   (5.29)  
race_hispanic    5.39       -1.18    10.83** 
                 (3.64)     (4.85)   (5.27)  
college_grad     11.83***   5.85     13.49***
                 (2.68)     (4.71)   (3.32)  
ideo_liberal     31.34***   14.09**  39.92***
                 (3.99)     (6.16)   (5.16)  
ideo_moderate    10.33***   10.55**  10

#### To assess potential multicollinearity among the independent variables, we compute Variance Inflation Factors (VIF). All five political-anxiety pillars exhibit VIF values below 4, indicating low collinearity. The age control variable has a moderately higher VIF (≈ 5.7), which reflects natural correlation between demographics and issue attitudes, but it remains well below the conventional thresholds (10) for concern.

#### Conclusion: There is no evidence of problematic multicollinearity, and all variables can be reliably included in the regression model.

### Omitted Variable Bias (OVB) Test

In [ ]:
#  Omitted Variable Bias (OVB) Check

# --- Model A: without controls ---
X_A = df_ind[[
    "inflation_stress",
    "border_stance",
    "abort_crime_view",
    "trans_support",
    "ukraine_support"
]].dropna()
y_A = df_ind.loc[X_A.index, "cand_biden"]

X_A = sm.add_constant(X_A)
model_A = sm.OLS(y_A, X_A).fit(cov_type="HC3")


# --- Model B: full model with controls ---
X_B = df_ind[[
    "inflation_stress",
    "border_stance",
    "abort_crime_view",
    "trans_support",
    "ukraine_support",
    "age",
    "female"
]].dropna()
y_B = df_ind.loc[X_B.index, "cand_biden"]

X_B = sm.add_constant(X_B)
model_B = sm.OLS(y_B, X_B).fit(cov_type="HC3")


# --- Try Stargazer, otherwise print both ---
try:
    from stargazer.stargazer import Stargazer
    stargazer_compare = Stargazer([model_A, model_B])
    display(HTML(stargazer_compare.render_html()))
except:
    print("Model A (without controls):")
    display(model_A.summary())
    print("\nModel B (with controls):")
    display(model_B.summary())


#### To assess the possibility of omitted variable bias, we estimate two models:

- ####  **Model (1):** Includes only the Five Pillars of political anxiety  
- ####  **Model (2):** Adds demographic controls (age and gender)

### A comparison of the two models shows that the coefficients on all five pillars are **extremely stable** across specifications. In particular:

- #### Inflation stress, border stance, and abortion views remain small and statistically insignificant in both models.  
- #### The two strongest predictors—**transgender rights attitudes** and **support for Ukraine aid**—retain nearly identical magnitudes and remain strongly statistically significant.  
- #### The model’s explanatory power (**R² = 0.318**) does not change when controls are added.  
- #### No changes in sign, magnitude, or statistical significance occur for any variable of interest.

#### **Interpretation:**  
#### The stability of the coefficients implies that adding demographic controls does **not** substantially alter the estimated effects of the Five Pillars. Therefore, omitted variable bias from basic demographics (age, gender) is unlikely to be driving the core results. While factors such as income, education, or partisanship are not included, the consistency across specifications suggests that the main findings—particularly the central role of cultural and foreign-policy anxieties—are **robust to alternative model specifications**.

#### The OVB check strengthens the conclusion that Independents’ evaluations of Biden are shaped more by cultural and global issue orientations than by material or security concerns, even after accounting for demographic differences.


## 5. Conclusion

### This study provides clear evidence that among Independent voters, cultural and foreign-policy attitudes are substantially more influential in shaping favorability toward President Biden than traditional economic or security concerns. Key findings include:

- ### **Cultural attitudes** — Support for transgender rights is one of the strongest negative predictors of Biden favorability, indicating that cultural value alignment plays a major role for Independents.
- ### **Foreign-policy views** — Support for U.S. aid to Ukraine is a strong positive predictor, suggesting that geopolitical orientations meaningfully shape evaluations of the incumbent.
- ### **Economic and material concerns** — Inflation stress, border-security preferences, and abortion-related views show weak or statistically insignificant relationships with Biden support.

### Although the analysis is cross-sectional and limited to a relatively small sample of self-identified Independents, the findings highlight that value-based and global issue orientations—not pocketbook or security anxieties—are central to how Independents evaluate President Biden.

### Discussion & Future Work

#### The results of this analysis raise important questions about how Independents form political attitudes and what issues truly drive their evaluations of presidential performance. The finding that cultural and foreign-policy concerns outweigh economic or border-security anxieties challenges common assumptions about voter behavior, especially in an election cycle where inflation and immigration dominate public discourse. One interpretation is that voters driven primarily by economic or security issues may have already sorted into stable partisan identities, leaving true Independents more responsive to value-based or global issues that signal broader ideological alignment.

#### Another possibility is that Independents who remain undecided tend to have relatively stable economic conditions, making symbolic or identity-related issues more salient than immediate material concerns. Alternatively, cultural and foreign-policy attitudes may serve as broader proxies for underlying worldviews—cosmopolitanism vs. nationalism, progressive vs. conservative identity—that shape how voters interpret the performance of political leaders.

#### **Future Work**

#### Several avenues for further research could extend and deepen these findings:

- #### **Expand the set of controls.** Including education, income, occupation, and especially party “leaners” would clarify whether the observed patterns persist when socioeconomic and ideological variables are incorporated.
- #### **Use longitudinal or panel data.** Tracking the same individuals over time would help determine whether cultural and foreign-policy concerns causally influence candidate evaluations, or whether shifting favorability leads to changes in issue attitudes.
- #### **Test different voter subgroups.** Comparing Independents to Democrats, Republicans, or “Independent leaners” could reveal whether the importance of cultural/global issues is unique to true Independents.
- #### **Incorporate additional political anxiety dimensions.** Issues such as climate change, crime, or health policy may provide more comprehensive measures of the anxiety landscape shaping Independent voter behavior.

 #### In summary, expanding the dataset, incorporating richer controls, and employing more robust causal methods would allow future research to more definitively determine why cultural and foreign-policy concerns play such an outsized role in shaping Independent voters’ evaluations of President Biden.

---
---

## 6. References
- **American National Election Studies.** (2024). *ANES 2024 Pilot Study*. Retrieved from https://electionstudies.org/data-center/2024-pilot-study/

- **White House.** (2022). *Fact Sheet: President Biden Announces Student Loan Relief for Borrowers Who Need It Most.* Biden White House Archives. https://bidenwhitehouse.archives.gov/briefing-room/statements-releases/2022/08/24/fact-sheet-president-biden-announces-student-loan-relief-for-borrowers-who-need-it-most/